In [1]:
import numpy as np
f1 = open(".csv","rb")
features =np.loadtxt(f1,delimiter=',',skiprows=0)
f1.close()
features_selected= np.array(features)
#features_selected= np.column_stack((k_mer,DSA)) 
print(features_selected.shape)

(1484, 1024)


In [2]:
# Read labels
target= np.load("/target.npy")
print(target.shape)

(1484,)


from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE(random_state=42).fit_sample(features_selected, target)
features_selected = X_resampled_smote
target = y_resampled_smote
print(features_selected.shape)
print(target.shape)

In [3]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

data = features_selected
y = target
#params={'gamma':np.logspace(-15,5,base=2),'C':np.logspace(-5,15,base=2)}
X_train,X_test,y_train,y_test=train_test_split(data,y,random_state=42)


#SVM
svc=SVC(kernel='rbf')
grid_search=GridSearchCV(svc,params,cv=5,n_jobs=-1)
grid_search.fit(X_train,y_train)
print('模型最优参数：{}'.format(grid_search.best_params_))
print('训练集最高分：{:.4f}'.format(grid_search.best_score_))
print('测试集最高分：{:.4f}'.format(grid_search.score(X_test,y_test)))

In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([("scaler", MinMaxScaler()), 
                 ("svm",SVC(decision_function_shape="ovo",kernel="rbf",probability = True,random_state = 21))])

param_grid = {"svm__C":[pow(2, k) for k in range(-5, 15)],
             "svm__gamma":[pow(2, k) for k in range(-15, 5)]}

# When we select 11220 features, the maximum overall accuracy can reach 88.82%
grid_search = GridSearchCV(pipe, param_grid, cv=10,n_jobs = -1)
grid_search.fit(features,target)
# The best values for parameters C and gamma.
print(grid_search.best_params_)
print(grid_search.best_score_)

{'svm__C': 1, 'svm__gamma': 0.0625}
0.6495918737529476


In [27]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [28]:
#高斯朴素贝叶斯
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
#gnb.fit(X_train,y_train)
#print('训练集最高分：{:.4f}'.format(gnb.score(X_train,y_train)))
#print('测试集最高分：{:.4f}'.format(gnb.score(X_test,y_test)))
print('得分： {:.4f}'.format(np.mean(cross_val_score(gnb, features, target, cv=10,n_jobs=-1))))

得分： 0.5883


In [29]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=21)
print('最高得分： {:.4f}'.format(np.mean(cross_val_score(dt, features, target, cv=10,n_jobs=-1))))

最高得分： 0.5431


In [30]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
params={'n_neighbors':list(range(1,100,5)),'leaf_size':list(range(1,100,5))}
knn=KNeighborsClassifier()
grid_search=GridSearchCV(knn,params,cv=10,n_jobs=-1)
#grid_search.fit(X_train,y_train)
#print('模型最优参数：{}'.format(grid_search.best_params_))
#print('训练集最高分：{:.4f}'.format(grid_search.best_score_))
#print('测试集最高分：{:.4f}'.format(grid_search.score(X_test,y_test)))
grid_search.fit(features, target)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'leaf_size': 1, 'n_neighbors': 96}
0.6138445492472338


In [31]:
#随机森林
from sklearn.ensemble import RandomForestClassifier
params={'n_estimators':[int(x) for x in np.arange(10,150,5)]}
rf=RandomForestClassifier(random_state=28,max_features='auto')
grid_search=GridSearchCV(rf,params,cv=10,n_jobs=-1)
#grid_search.fit(X_train,y_train)
#print('模型最优参数：{}'.format(grid_search.best_params_))
#print('训练集最高分：{:.4f}'.format(grid_search.best_score_))
#print('测试集最高分：{:.4f}'.format(grid_search.score(X_test,y_test)))
grid_search.fit(features, target)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'n_estimators': 45}
0.6051151822963904


In [15]:
#Bagging
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(random_state = 21)
params={'n_estimators':[int(x) for x in np.arange(5,200,5)]}
grid_search=GridSearchCV(bagging,params,cv=10,n_jobs=-1)
#grid_search.fit(X_train,y_train)
#print('模型最优参数：{}'.format(grid_search.best_params_))
#print('训练集最高分：{:.4f}'.format(grid_search.best_score_))
#print('测试集最高分：{:.4f}'.format(grid_search.score(X_test,y_test)))
#print('得分： {:.4f}'.format(np.mean(cross_val_score(clf, features_selected, target, cv=5,n_jobs=-1))))
grid_search.fit(features_selected, target)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'n_estimators': 130}
0.7479536354536355


In [16]:
#AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier ()
params={'n_estimators':[int(x) for x in np.arange(5,200,5)]}
grid_search=GridSearchCV(adaboost,params,cv=10,n_jobs=-1)
#grid_search.fit(X_train,y_train)
#print('模型最优参数：{}'.format(grid_search.best_params_))
#print('训练集最高分：{:.4f}'.format(grid_search.best_score_))
#print('测试集最高分：{:.4f}'.format(grid_search.score(X_test,y_test)))
grid_search.fit(features_selected, target)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'n_estimators': 135}
0.7455944580944581
